## Alpha-carbon distance change calculator
**Author:** Diego del Alamo

**Date:** 1 May 2021

**Description:** Checks the largest distance change between alpha carbons in two protein models (provided by PDB files) across a range of residues

This program quickly determines the residue pair undergoing the largest amplitude distance change between two protein structures. Distances between alpha carbons are measured. This can be used to quickly determine which residues to spin label for pulse EPR measurements.

**TODO:** Determine distances using spin labels rather than alpha carbons.

In [13]:
import numpy as np
import itertools
import ipywidgets as widgets

PDB file #1:

In [2]:
uploader1 = widgets.FileUpload( accept='.pdb', multiple=False )
display( uploader1 )

FileUpload(value={}, accept='.pdb', description='Upload')

PDB file #2:

In [3]:
uploader2 = widgets.FileUpload( accept='.pdb', multiple=False )
display( uploader2 )

FileUpload(value={}, accept='.pdb', description='Upload')

Restraints to check (comma-separated, with residues separated by dashes):


In [4]:
default = "195A-195B, 196A-196B, 197A-197B, 198A-198B"
pairs_in = widgets.Textarea( value=default,
                          placeholder=default,
                          description="Residue pairs",
                          disabled=False )
display( pairs_in )

Textarea(value='195A-195B, 196A-196B, 197A-197B, 198A-198B', description='Residue pairs', placeholder='195A-19…

Minimum and maximum distance (in angstroms):

In [5]:
lower_upper = widgets.FloatRangeSlider(
    value=[ 15., 60. ],
    min=0,
    max=100,
    step=1.,
    description="Distances (Å)",
    disabled=False,
    orientation="horizontal",
    readout=True
)
display( lower_upper )

FloatRangeSlider(value=(15.0, 60.0), description='Distances (Å)', step=1.0)

In [6]:
def process( line ):
    resn = int( line[ 22: 27 ] )
    chain = line[ 21 ]
    x = float( line[ 28:39 ] )
    y = float( line[ 39:47 ] )
    z = float( line[ 47:55 ] )
    return str( resn ) + chain, ( x, y, z )

In [7]:
def dist( xyz1, xyz2 ):
    return np.sqrt( sum([ ( xyz1[ i ] - xyz2[ i ]) ** 2 for i in range( 3 ) ]))

In [22]:
out = widgets.Output()
@out.capture( clear_output=False, wait=True )
def calculate_everything( _ ):
    
    lower = lower_upper.value[ 0 ]
    upper = lower_upper.value[ 1 ]
    
    pairs = [ pair.strip().split( "-" ) for pair in pairs_in.value.split( "," ) ]
    residues = set( itertools.chain( [ x[ 0 ] for x in pairs ], [ x[ 1 ] for x in pairs ] ) )
    
    model1_pair_dists = dict()
    model2_pair_dists = dict()
    for uploader, dists in [ ( uploader1, model1_pair_dists ),
                              ( uploader2, model2_pair_dists ) ]:
        coords = dict()
        for line in str( uploader.data[ 0 ] ).split( "\\n" ):
            if line.startswith( "ATOM" ) and " CA " in line:
                res, xyz = process( line )
                if res in residues:
                    coords[ res ] = xyz
        for res in residues:
            if res not in coords:
                print( "WARNING: {} not found in PDB file!".format( res ) )
        for pair in pairs:
            dists[ tuple( pair ) ] = dist( coords[ pair[ 0 ] ], coords[ pair[ 1 ] ] )


    best = None
    dists = None
    ddist = None
    for pair in pairs:
        dist1, dist2 = model1_pair_dists[ tuple( pair ) ], model2_pair_dists[ tuple( pair ) ]
        if dist1 < lower or dist1 > upper or dist2 < lower or dist2 > upper:
            continue
        temp_ddist = abs( dist1 - dist2 )
        if ddist is None or temp_ddist > ddist:
            best = pair
            dists = ( dist1, dist2 )
            ddist = temp_ddist
    if best is None:
        print( "ERROR: No distances satisfying the parameters were found!" )
    else:
        print( "Best pair: {}-{}".format( best[ 0 ], best[ 1 ] ) )
        print( "CA distance in model 1: {}".format( dists[ 0 ] ) )
        print( "CA distance in model 2: {}".format( dists[ 1 ] ) )

In [23]:
whatever = ""
go = widgets.Button(
    value=False,
    description="Calculate" )
display( go )
def nullfxn():
    return False
go.on_click( calculate_everything )
out

Button(description='Calculate', style=ButtonStyle())

Output()